<a href="https://colab.research.google.com/github/RashmikaReddy/Contrastive_Learning_Implementation/blob/main/MocoV2Implementation_Final_Github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Useful imports
# Getting the dataset from the github repo of imagenet 5 categories
!git clone https://github.com/thunderInfy/imagenet-5-categories.git

import numpy as np
import torch
import copy
from torchvision import transforms as T
from torchsummary import summary
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18
import os
import glob
from PIL import Image
from collections import OrderedDict
import random
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import seaborn as sns
tsne = TSNE()



Cloning into 'imagenet-5-categories'...
remote: Enumerating objects: 1532, done.
remote: Total 1532 (delta 0), reused 0 (delta 0), pack-reused 1532
Receiving objects: 100% (1532/1532), 88.56 MiB | 13.91 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# device is set to cuda if cuda is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# root folder is the name of the folder where data is contained
#root_folder = '/content/drive/MyDrive/imagenet-5-categories'
root_folder = 'imagenet-5-categories'

train_names = sorted(os.listdir(root_folder + '/train'))
test_names = sorted(os.listdir(root_folder + '/test'))

# setting random seed to ensure the same 10% labelled data is used when training the linear classifier
random.seed(0)

names_train_10_percent = random.sample(train_names, len(train_names) // 10)
names_train = random.sample(train_names, len(train_names))
names_test = random.sample(test_names, len(test_names))


In [ ]:
print(names_train_10_percent)
print(names_train)
print(names_test)

['dog_1407995394_22d8274d95.jpg', 'dog_298325020_925ce65f7c.jpg', 'airplane_2110882407_db1564e294.jpg', 'cat_130028450_3bea7e5120.jpg', 'elephant_1476324821_d53eeef43f.jpg', 'dog_959590073_b411321069.jpg', 'dog_2021522094_e3998b18e0.jpg', 'cat_299837288_5c84203463.jpg', 'dog_690883918_0a2ebf2d4a.jpg', 'cat_809663020_7ce9d0899f.jpg', 'elephant_518813412_12505cf38a.jpg', 'car_2666463976_54eedf3bb0.jpg', 'elephant_1371597085_668b079f8d.jpg', 'car_2060217143_9725741a28.jpg', 'cat_201964852_1817e956c1.jpg', 'car_2060326215_d0289b872e.jpg', 'airplane_388364587_64b2370a82.jpg', 'cat_1130458060_b77117482a.jpg', 'elephant_1919497998_3ca0e4b024.jpg', 'elephant_81317196_a96223ef04.jpg', 'car_2224497766_1de83768a7.jpg', 'cat_371068935_af213116ce.jpg', 'airplane_420257932_91a79f131f.jpg', 'airplane_257796960_f2b410e9cd.jpg', 'cat_495308681_e317861b3e.jpg', 'dog_515415369_5b5258ea94.jpg', 'elephant_342367081_234bf66913.jpg', 'airplane_425399322_b8c9f3facf.jpg', 'cat_797614305_bec61c42af.jpg', 'dog_3

In [ ]:
# defining a mapping between class names and numbers
mapping = {'car': 0, 'dog': 1, 'elephant': 2, 'cat': 3, 'airplane': 4}
inverse_mapping = ['car', 'dog', 'elephant', 'cat', 'airplane']

# getting labels based on filenames, note that the filenames themselves contain classnames
# also note that these labels won't be used to actually train the base model
# these are just for visualization purposes
labels_train = [mapping[x.split('_')[0]] for x in names_train]
labels_test = [mapping[x.split('_')[0]] for x in names_test]

# these 10 percent labels will be used for training the linear classifer
labels_train_10_percent = [mapping[x.split('_')[0]] for x in names_train_10_percent]


In [ ]:
print(labels_train)
print(labels_test)
print(labels_train_10_percent)

[4, 4, 0, 0, 2, 2, 4, 1, 4, 1, 4, 4, 4, 0, 0, 0, 1, 0, 4, 4, 2, 1, 4, 3, 4, 0, 4, 3, 3, 1, 0, 2, 2, 1, 4, 4, 2, 0, 2, 3, 1, 2, 0, 0, 4, 0, 0, 3, 2, 3, 4, 1, 0, 4, 1, 1, 2, 2, 3, 3, 1, 2, 0, 2, 4, 1, 4, 3, 4, 2, 3, 0, 0, 1, 3, 3, 3, 0, 3, 1, 1, 4, 4, 0, 3, 0, 0, 0, 1, 1, 2, 4, 1, 1, 4, 0, 1, 4, 3, 0, 1, 2, 1, 4, 4, 2, 3, 3, 2, 4, 1, 0, 2, 4, 0, 4, 1, 1, 3, 4, 0, 4, 4, 2, 4, 0, 4, 0, 3, 3, 0, 4, 1, 1, 4, 2, 3, 1, 2, 3, 0, 2, 3, 4, 0, 3, 4, 4, 4, 0, 3, 3, 1, 2, 2, 1, 1, 1, 0, 0, 1, 3, 4, 0, 0, 3, 3, 3, 1, 4, 3, 4, 4, 3, 0, 0, 3, 3, 1, 0, 3, 2, 1, 2, 4, 3, 0, 4, 2, 1, 3, 3, 1, 4, 4, 1, 1, 3, 3, 2, 2, 2, 2, 1, 4, 3, 3, 0, 0, 2, 4, 4, 4, 0, 2, 2, 1, 4, 2, 1, 3, 1, 2, 0, 1, 4, 1, 4, 2, 2, 3, 2, 2, 0, 4, 2, 4, 4, 1, 1, 2, 3, 4, 1, 3, 0, 1, 3, 2, 4, 3, 0, 0, 4, 3, 4, 2, 1, 4, 3, 4, 4, 4, 0, 0, 0, 3, 1, 4, 1, 3, 0, 2, 3, 0, 3, 3, 0, 1, 3, 3, 3, 2, 1, 0, 1, 3, 2, 3, 2, 1, 2, 2, 3, 1, 4, 4, 0, 3, 1, 1, 0, 1, 0, 4, 3, 1, 1, 0, 3, 1, 4, 1, 0, 2, 0, 2, 3, 1, 0, 4, 1, 2, 3, 1, 4, 1, 4, 4, 4, 0, 4, 1, 

In [ ]:
def get_color_distortion(s=1.0):
    color_jitter = T.ColorJitter(0.8 * s, 0.8 * s, 0.8 * s, 0.2 * s)
    rnd_color_jitter = T.RandomApply([color_jitter], p=0.8)
    
    # p is the probability of grayscale, here 0.2
    rnd_gray = T.RandomGrayscale(p=0.2)
    color_distort = T.Compose([rnd_color_jitter, rnd_gray])
    
    return color_distort

# this is the dataset class
class MyDataset(Dataset):
    def __init__(self, filenames, labels, mutation=False):
        self.file_names = filenames
        self.labels = labels
        self.mutation = mutation

    def __len__(self):
        return len(self.file_names)

    def tensorify(self, img):
        res = T.ToTensor()(img)
        res = T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))(res)
        return res

    def mutate_image(self, img):
        res = T.RandomResizedCrop(224)(img)
        res = get_color_distortion()(res)
        return res

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = self.file_names[idx]
        image = Image.open(img_name).convert('RGB')
        label = self.labels[idx]
        image = T.Resize((250, 250))(image)

        if self.mutation:
            image1 = self.mutate_image(image)
            image1 = self.tensorify(image1)
            image2 = self.mutate_image(image)
            image2 = self.tensorify(image2)
            sample = {'image1': image1, 'image2': image2, 'label': label}
        else:
            image = T.Resize((224, 224))(image)
            image = self.tensorify(image)
            sample = {'image': image, 'label': label}

        return sample

In [ ]:
# datasets
training_dataset_mutated = MyDataset(names_train, labels_train, mutation=True)
training_dataset = MyDataset(names_train_10_percent, labels_train_10_percent, mutation=False)
testing_dataset = MyDataset(names_test, labels_test, mutation=False)

# dataloaders
dataloader_training_dataset_mutated = DataLoader(training_dataset_mutated, batch_size=256, shuffle=True)
dataloader_training_dataset = DataLoader(training_dataset, batch_size=256, shuffle=True)
dataloader_testing_dataset = DataLoader(testing_dataset, batch_size=256, shuffle=True)

In [ ]:
# defining our deep learning architecture
resnetq = resnet18(pretrained=False)

classifier = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(resnetq.fc.in_features, 100)),
    ('added_relu1', nn.ReLU(inplace=True)),
    ('fc2', nn.Linear(100, 50)),
    ('added_relu2', nn.ReLU(inplace=True)),
    ('fc3', nn.Linear(50, 25))
]))

resnetq.fc = classifier
#Finally, resnetk is created as a deep copy of resnetq using the copy.deepcopy() method. 
#This is done to ensure that resnetk has its own set of parameters and is not simply a reference to resnetq.
resnetk = copy.deepcopy(resnetq)

# moving the resnet architecture to device
resnetq.to(device)
resnetk.to(device)


/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
τ = 0.05

def loss_function(q, k, queue):
    N = q.shape[0]
    C = q.shape[1]

    pos = torch.exp(torch.div(torch.bmm(q.view(N,1,C), k.view(N,C,1)).view(N, 1),τ))
    neg = torch.sum(torch.exp(torch.div(torch.mm(q.view(N,C), torch.t(queue)),τ)), dim=1)
    denominator = neg + pos

    return torch.mean(-torch.log(torch.div(pos,denominator)))


In [ ]:
type(dataloader_training_dataset_mutated)

torch.utils.data.dataloader.DataLoader

In [ ]:
for i,j in enumerate(dataloader_training_dataset_mutated):
  print("i is ",i)
  print("data is",j)

FileNotFoundError: ignored

In [ ]:

K = 1250

# Defining data structures for storing training info

losses_train = []
num_epochs = 0

flag = 0

queue = None

# using SGD optimizer
optimizer = optim.SGD(resnetq.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-6)

if not os.path.exists('/content/drive/MyDrive/MoCoImplementation/results'):
    os.makedirs('/content/drive/MyDrive/MoCoImplementation/results')

# # load pretrained model, optimizer and training losses file if model.pth file is available
# if(os.path.isfile("results/modelq.pth")):
#     resnetq.load_state_dict(torch.load("results/modelq.pth"))
#     resnetk.load_state_dict(torch.load("results/modelk.pth"))
#     optimizer.load_state_dict(torch.load("results/optimizer.pth"))

#     # for param_group in optimizer.param_groups:
#     #     param_group['weight_decay'] = 1e-6
#     #     param_group['lr'] = 0.00001

#     temp = np.load("results/lossesfile.npz")
#     losses_train = list(temp['arr_0'])
#     queue = torch.load("results/queue.pt")


if queue is None:
    while True:

        with torch.no_grad():
            for (_, sample_batched) in enumerate(dataloader_training_dataset_mutated):            
                print("inside loop")
                xk = sample_batched['image2']
                xk = xk.to(device)
                k = resnetk(xk)
                k = k.detach()

                k = torch.div(k,torch.norm(k,dim=1).reshape(-1,1))

                if queue is None:
                    queue = k
                else:
                    if queue.shape[0] < K:
                        queue = torch.cat((queue, k), 0)    
                    else:
                        flag = 1
                
                if flag == 1:
                    break

        if flag == 1:
            break


# Boolean variable on whether to perform training or not 

TRAINING = True
momentum = 0.999

def get_mean_of_list(L):
    return sum(L) / len(L)

if TRAINING:
    # get resnet in train mode
    resnetq.train()

    # run a for loop for num_epochs
    for epoch in range(num_epochs):

        print(epoch)

        # a list to store losses for each epoch
        epoch_losses_train = []

        # run a for loop for each batch
        for (_, sample_batched) in enumerate(dataloader_training_dataset_mutated):
            print('_ is ',_)
            print('sample_batched is ',sample_batched)            
            # zero out grads
            optimizer.zero_grad()

            # retrieve xq and xk the two image batches
            xq = sample_batched['image1']
            xk = sample_batched['image2']

            # move them to the device
            xq = xq.to(device)
            xk = xk.to(device)

            # get their outputs
            q = resnetq(xq)
            k = resnetk(xk)
            k = k.detach()

            q = torch.div(q,torch.norm(q,dim=1).reshape(-1,1))
            k = torch.div(k,torch.norm(k,dim=1).reshape(-1,1))

            # get loss value
            loss = loss_function(q, k, queue)
            
            # put that loss value in the epoch losses list
            epoch_losses_train.append(loss.cpu().data.item())

            # perform backprop on loss value to get gradient values
            loss.backward()

            # run the optimizer
            optimizer.step()

            # update the queue
            queue = torch.cat((queue, k), 0) 

            if queue.shape[0] > K:
                queue = queue[256:,:]

            # update resnetk
            for θ_k, θ_q in zip(resnetk.parameters(), resnetq.parameters()):
                θ_k.data.copy_(momentum*θ_k.data + θ_q.data*(1.0 - momentum))

        # append mean of epoch losses to losses_train, essentially this will reflect mean batch loss
        losses_train.append(get_mean_of_list(epoch_losses_train))

        %cd simclr-with-momentum

        # Plot the training losses Graph and save it
        fig = plt.figure(figsize=(10, 10))
        sns.set_style('darkgrid')
        plt.plot(losses_train)
        plt.legend(['Training Losses'])
        plt.savefig('/content/drive/MyDrive/MoCoImplementation/results/losses.png')
        plt.close()

        # Store model and optimizer files
        torch.save(resnetq.state_dict(), '/content/drive/MyDrive/MoCoImplementation/results/modelq.pth')
        torch.save(resnetk.state_dict(), '/content/drive/MyDrive/MoCoImplementation/results/modelk.pth')
        torch.save(optimizer.state_dict(), '/content/drive/MyDrive/MoCoImplementation/results/optimizer.pth')
        np.savez("/content/drive/MyDrive/MoCoImplementation/results/lossesfile", np.array(losses_train))
        torch.save(queue, '/content/drive/MyDrive/MoCoImplementation/results/queue.pt')
        %cd ..


FileNotFoundError: ignored

In [ ]:
import statistics

# removing the projection head
if len(nn.Sequential(*list(resnetq.fc.children()))) == 5:
    resnetq.fc = nn.Sequential(*list(resnetq.fc.children())[:-3])


Classes_List = sorted(os.listdir('imagenette/train'))
Classes_Map = dict()
for i in range(len(Classes_List)):
    Classes_Map[Classes_List[i]] = i

# Boolean variable to control whether to train the linear classifier or not
LINEAR = True

class LinearNet(nn.Module):

    def __init__(self):
        super(LinearNet, self).__init__()
        self.fc1 = torch.nn.Linear(100, 10)

    def forward(self, x):
        x = self.fc1(x)
        return(x)

if LINEAR:

    if not os.path.exists('/content/drive/MyDrive/MoCoImplementation/linear'):
        os.makedirs('/content/drive/MyDrive/MoCoImplementation/linear')

    # getting our linear classifier
    linear_classifier = LinearNet()

    # moving it to device
    linear_classifier.to(device)

    # using SGD as a linear optimizer
    linear_optimizer = optim.SGD(linear_classifier.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-6)

    #number of epochs
    num_epochs_linear = 20

    # Boolean variable to control training of linear classifier
    LINEAR_TRAINING = True

    # Defining data structures to store train and test info for linear classifier
    losses_train_linear = []
    acc_train_linear = []
    losses_test_linear = []
    acc_test_linear = []
    test_accuracy = []


    # a variable to keep track of the maximum test accuracy, will be useful to store 
    # model parameters with the best test accuracy
    max_test_acc = 0

    # if a model exists in the linear folder, load it
    if(os.path.isfile("/content/drive/MyDrive/MoCoImplementation/linear/model.pth")):

        # load state dict for linear model and optimizer
        linear_classifier.load_state_dict(torch.load("/content/drive/MyDrive/MoCoImplementation/linear/model.pth"))
        linear_optimizer.load_state_dict(torch.load("/content/drive/MyDrive/MoCoImplementation/linear/optimizer.pth"))

        # change learning rate, you can change its values if you don't feel its necessity while training
        # for g in linear_optimizer.param_groups:
        #   g['lr'] = 0.001
        #   g['weight_decay'] = 0

        # load data structures
        # 

    # Run a for loop for training the linear classifier
    for epoch in range(num_epochs_linear):

        if LINEAR_TRAINING:

            # run linear classifier in train mode
            linear_classifier.train()

            # a list to store losses for each batch in an epoch
            epoch_losses_train_linear = []
            epoch_acc_train_num_linear = 0.0
            epoch_acc_train_den_linear = 0.0

            # for loop for running through each batch
            for (_, sample_batched) in enumerate(dataloader_training_dataset):

                # get x and y from the batch
                x = sample_batched['image']
                y_actual = sample_batched['label']
                y_actual = torch.tensor([Classes_Map[i] for i in y_actual])

                # move them to the device
                x = x.to(device)
                y_actual  = y_actual.to(device)

                with torch.no_grad():
                    # get output from resnet architecture
                    y_intermediate = resnetq(x)

                # zero the grad values
                linear_optimizer.zero_grad()

                # run y_intermediate through the linear classifier
                y_predicted = linear_classifier(y_intermediate)

                # get the cross entropy loss value
                loss = nn.CrossEntropyLoss()(y_predicted, y_actual)

                # add the obtained loss value to this list
                epoch_losses_train_linear.append(loss.data.item())
                
                # perform backprop through the loss value
                loss.backward()

                # call the linear_optimizer step function
                linear_optimizer.step()

                # get predictions and actual values to cpu  
                pred = np.argmax(y_predicted.cpu().data, axis=1)
                actual = y_actual.cpu().data

                #update the numerators and denominators of accuracy
                epoch_acc_train_num_linear += (actual == pred).sum().item()
                epoch_acc_train_den_linear += len(actual)

                x = None
                y_intermediate = None
                y_predicted = None
                sample_batched = None

            # update losses and acc lists    
            losses_train_linear.append(get_mean_of_list(epoch_losses_train_linear))
            acc_train_linear.append(epoch_acc_train_num_linear / epoch_acc_train_den_linear)
        
        # run linear classifier in eval mode
        linear_classifier.eval()

        # essential variables to keep track of losses and acc
        epoch_losses_test_linear = []
        epoch_acc_test_num_linear = 0.0
        epoch_acc_test_den_linear = 0.0

        # run a for loop through each batch
        for (_, sample_batched) in enumerate(dataloader_testing_dataset):
            x = sample_batched['image']
            y_actual = sample_batched['label']
            y_actual = torch.tensor([Classes_Map[i] for i in y_actual])

            x = x.to(device)
            y_actual  = y_actual.to(device)

            with torch.no_grad():
                y_intermediate = resnetq(x)

            y_predicted = linear_classifier(y_intermediate)
            loss = nn.CrossEntropyLoss()(y_predicted, y_actual)
            epoch_losses_test_linear.append(loss.data.item())

            pred = np.argmax(y_predicted.cpu().data, axis=1)
            actual = y_actual.cpu().data
            epoch_acc_test_num_linear += (actual == pred).sum().item()
            epoch_acc_test_den_linear += len(actual)

        # calculate test_acc
        test_acc = epoch_acc_test_num_linear / epoch_acc_test_den_linear
        print("test accuracy for epoch ",epoch," is ",test_acc)
        test_accuracy.append(test_acc)

        if LINEAR_TRAINING:
            losses_test_linear.append(get_mean_of_list(epoch_losses_test_linear))
            acc_test_linear.append(epoch_acc_test_num_linear / epoch_acc_test_den_linear)

            # plotting losses and accuracies

            fig = plt.figure(figsize=(10, 10))
            sns.set_style('darkgrid')
            plt.plot(losses_train_linear)
            plt.plot(losses_test_linear)
            plt.legend(['Training Losses', 'Testing Losses'])
            plt.savefig('/content/drive/MyDrive/MoCoImplementation/linear/losses.png')
            plt.close()

            fig = plt.figure(figsize=(10, 10))
            sns.set_style('darkgrid')
            plt.plot(acc_train_linear)
            plt.plot(acc_test_linear)
            plt.legend(['Training Accuracy', 'Testing Accuracy'])
            plt.savefig('/content/drive/MyDrive/MoCoImplementation/linear/accuracy.png')
            plt.close()

            print("Epoch completed")

            if test_acc >= max_test_acc:

                # save the model only when test_acc exceeds the current max_test_acc

                max_test_acc = test_acc
                torch.save(linear_classifier.state_dict(), '/content/drive/MyDrive/MoCoImplementation/linear/model.pth')
                torch.save(linear_optimizer.state_dict(), '/content/drive/MyDrive/MoCoImplementation/linear/optimizer.pth')

        # save data structures
        np.savez("/content/drive/MyDrive/MoCoImplementation/linear/linear_losses_train_file", np.array(losses_train_linear))
        np.savez("/content/drive/MyDrive/MoCoImplementation/linear/linear_losses_test_file", np.array(losses_test_linear))
        np.savez("/content/drive/MyDrive/MoCoImplementation/linear/linear_acc_train_file", np.array(acc_train_linear))
        np.savez("/content/drive/MyDrive/MoCoImplementation/linear/linear_acc_test_file", np.array(acc_test_linear))
    print("Testing accuracy for self-supervised learning using MoCo-v2 is ",statistics.mean(test_accuracy))


test accuracy for epoch  0  is  0.11617834394904458
Epoch completed
test accuracy for epoch  1  is  0.18369426751592358
Epoch completed
test accuracy for epoch  2  is  0.24076433121019108
Epoch completed
test accuracy for epoch  3  is  0.2736305732484076
Epoch completed
test accuracy for epoch  4  is  0.3470063694267516
Epoch completed
test accuracy for epoch  5  is  0.4356687898089172
Epoch completed
test accuracy for epoch  6  is  0.4045859872611465
Epoch completed
test accuracy for epoch  7  is  0.3915923566878981
Epoch completed
test accuracy for epoch  8  is  0.4043312101910828
Epoch completed
test accuracy for epoch  9  is  0.40662420382165604
Epoch completed
test accuracy for epoch  10  is  0.43006369426751595
Epoch completed
test accuracy for epoch  11  is  0.49044585987261147
Epoch completed
